In [1]:
# external api
import fundamentalanalysis as fa
import yfinance as yf

# data analytic 
import numpy as np
import pandas as pd

# additional liberies
import datetime
import statistics as st

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
api_key = '00ef9804fcde0edd93b1b4821ee2f06a'  #job.chap@icloud.com
# api_key = 'c81352430e2fe3c941faf0814227562b'  #jobs.chaps@gmail.com
# api_key = 'a765d11740cccfb61177da0ad8699d1e'  #job.chap@gmail.com

In [4]:
''' Тикеры '''
tick = 'F'  # поменял название переменно ticker на tick, для настройки по одной акции

''' для фильтрации'''
# filt_start_date = '2000'
# filt_finish_date = '2022'


''' время '''
stime = '2000-01-01'
ftime = '2023-02-01'
period = '1mo'  # '1d' '1wk'

In [5]:
stime[:4]

'2000'

In [6]:
class Fund_ticker:
    def __init__(self, tick, stime, ftime, api_key_fund):
        self.tick = tick
        self.stime = stime
        self.ftime = ftime
        self.api_key_fund = api_key_fund
        
        
    def _change_index_plus_one(self, df):
        """ добавлят цифру к году в фундаментальные api, чтобы видень данные как начало года, а не как факт 
        """
        
        df_index = df.index.to_list()
        
        for i in range(len(df_index)):
            df_index[i] = str(int(df_index[i])+1)
            
        df['Date_index'] = df_index
        
        df = df.set_index('Date_index')
        
        return df
        
    def get_key_metr(self):
        """ получаем df с key_metrics
        """
        
        df_key_metrics = fa.key_metrics(self.tick, self.api_key_fund, period='annual')
        df_key_metrics = self._change_index_plus_one(df_key_metrics.T)
        
        return df_key_metrics
    
    def get_income_statement(self):
        """ получаем df с income_statement
        """   
        
        income_statement = fa.income_statement(self.tick, self.api_key_fund, period='annual')
        income_statement = self._change_index_plus_one(income_statement.T)
        
        return income_statement
        
    
    def get_balance_sheet_statement(self):
        """ получаем df с balance_sheet_statement
        """  
        
        balance_sheet_statement = fa.balance_sheet_statement(self.tick, self.api_key_fund, period='annual')
        balance_sheet_statement = self._change_index_plus_one(balance_sheet_statement.T)
        
        return balance_sheet_statement

    def get_cash_flow_statement(self):   
        """ получаем df с cash_flow_statement
        """    
        
        cash_flow_statement = fa.cash_flow_statement(self.tick, self.api_key_fund, period='annual')
        cash_flow_statement = self._change_index_plus_one(cash_flow_statement.T)
        
        return cash_flow_statement

    def get_financial_ratios(self):    
        """ получаем df с financial_ratios
        """
        
        financial_ratios = fa.financial_ratios(self.tick, self.api_key_fund, period='annual')
        financial_ratios = self._change_index_plus_one(financial_ratios.T)
        
        return financial_ratios

    def get_financial_statement_growth(self):
        """ получаем df с financial_statement_growth
        """ 
        
        financial_statement_growth = fa.financial_statement_growth(self.tick, self.api_key_fund, period='annual')
        financial_statement_growth = self._change_index_plus_one(financial_statement_growth.T)
        
        return financial_statement_growth
    
    
    def all_metrics(self):
        """ df со всеми колонками = конкатенация колонок со всех полученных таблиц со значениями
        """
        
        df_km = self.get_key_metr()
        df_is = self.get_income_statement()
        df_bss = self.get_balance_sheet_statement()
        df_cfs = self.get_cash_flow_statement()
        df_fr = self.get_financial_ratios()
        df_fsg = self.get_financial_statement_growth()
        
        df_all_fa_columns = pd.concat([df_km,
                                       df_is,
                                       df_bss,
                                       df_cfs,
                                       df_fr,
                                       df_fsg
                                       ], axis=1)
        
        """ убраем дубликаты колонок """
        df_all_fa_columns = df_all_fa_columns.loc[:,~pd.concat([df_km,
                                                                df_is,
                                                                df_bss,
                                                                df_cfs,
                                                                df_fr,
                                                                df_fsg
                                                                ], axis=1).columns.duplicated()]
        
        
        """ удаляются не используемые колонки """
        df_all_fa_columns = df_all_fa_columns.drop(['period', 'reportedCurrency', 'cik', 'fillingDate', 'calendarYear', 'link', 'finalLink'], axis=1)        
        
        return df_all_fa_columns
    
    def all_metrics_period(self):
        """ df со всеми колонками отфильтрованная в нужный период
        """
        
        df = self.all_metrics()
        
        # берем из дат 4е символа
        start = self.stime[:4]
        finish = self.ftime[:4]
        
        df_filt = df.loc[finish:start].copy(deep=True)
        
        return df_filt
        
    def get_data_from_ticker(self):
        ''' получаем цены из yahoo_finance
        '''
        ticker = yf.Ticker(self.tick)

        df = ticker.history(start=self.stime, end=self.ftime, interval='1mo')
        x = pd.DataFrame(df)
        x.rename(columns={"Close": self.tick}, inplace=True)
        z = x.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"])
        
        return z

    def _change_index_yahoo(self):
        ''' сокращаем индекс из dataframe yahoo
        '''
        
        df = self.get_data_from_ticker()
        df_index = df.index.to_list()
        for i in range(len(df_index)):
            t = df_index[i].date()
            df_index[i] = t.strftime('%Y-%m-%d')
        df['Date_index'] = df_index
        df = df.set_index('Date_index')
        return df

        
    
    
        
    
        
    

In [7]:
# x = Fund_ticker(tick, stime, ftime, api_key).all_metrics_period()
x = Fund_ticker(tick, stime, ftime, api_key).all_metrics()

In [18]:
x.shape

(38, 234)

In [10]:
# Fund_ticker(tick, stime, ftime, api_key).get_key_metr()

In [11]:
x

,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,bookValuePerShare,tangibleBookValuePerShare,shareholdersEquityPerShare,interestDebtPerShare,marketCap,enterpriseValue,peRatio,priceToSalesRatio,pocfratio,pfcfRatio,pbRatio,ptbRatio,evToSales,enterpriseValueOverEBITDA,evToOperatingCashFlow,evToFreeCashFlow,earningsYield,freeCashFlowYield,debtToEquity,debtToAssets,netDebtToEBITDA,currentRatio,interestCoverage,incomeQuality,dividendYield,payoutRatio,salesGeneralAndAdministrativeToRevenue,researchAndDdevelopementToRevenue,intangiblesToTotalAssets,capexToOperatingCashFlow,capexToRevenue,capexToDepreciation,stockBasedCompensationToRevenue,grahamNumber,roic,returnOnTangibleAssets,grahamNetNet,workingCapital,tangibleAssetValue,netCurrentAssetValue,investedCapital,averageReceivables,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare,acceptedDate,revenue,costOfRevenue,grossProfit,grossProfitRatio,researchAndDevelopmentExpenses,generalAndAdministrativeExpenses,sellingAndMarketingExpenses,sellingGeneralAndAdministrativeExpenses,otherExpenses,operatingExpenses,costAndExpenses,interestIncome,interestExpense,depreciationAndAmortization,ebitda,ebitdaratio,operatingIncome,operatingIncomeRatio,totalOtherIncomeExpensesNet,incomeBeforeTax,incomeBeforeTaxRatio,incomeTaxExpense,netIncome,netIncomeRatio,eps,epsdiluted,weightedAverageShsOut,weightedAverageShsOutDil,cashAndCashEquivalents,shortTermInvestments,cashAndShortTermInvestments,netReceivables,inventory,otherCurrentAssets,totalCurrentAssets,propertyPlantEquipmentNet,goodwill,intangibleAssets,goodwillAndIntangibleAssets,longTermInvestments,taxAssets,otherNonCurrentAssets,totalNonCurrentAssets,otherAssets,totalAssets,accountPayables,shortTermDebt,taxPayables,deferredRevenue,otherCurrentLiabilities,totalCurrentLiabilities,longTermDebt,deferredRevenueNonCurrent,deferredTaxLiabilitiesNonCurrent,otherNonCurrentLiabilities,totalNonCurrentLiabilities,otherLiabilities,capitalLeaseObligations,totalLiabilities,preferredStock,commonStock,retainedEarnings,accumulatedOtherComprehensiveIncomeLoss,othertotalStockholdersEquity,totalStockholdersEquity,totalEquity,totalLiabilitiesAndStockholdersEquity,minorityInterest,totalLiabilitiesAndTotalEquity,totalInvestments,totalDebt,netDebt,deferredIncomeTax,stockBasedCompensation,changeInWorkingCapital,accountsReceivables,accountsPayables,otherWorkingCapital,otherNonCashItems,netCashProvidedByOperatingActivities,investmentsInPropertyPlantAndEquipment,acquisitionsNet,purchasesOfInvestments,salesMaturitiesOfInvestments,otherInvestingActivites,netCashUsedForInvestingActivites,debtRepayment,commonStockIssued,commonStockRepurchased,dividendsPaid,otherFinancingActivites,netCashUsedProvidedByFinancingActivities,effectOfForexChangesOnCash,netChangeInCash,cashAtEndOfPeriod,cashAtBeginningOfPeriod,operatingCashFlow,capitalExpenditure,freeCashFlow,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,daysOfPayablesOutstanding,cashConversionCycle,grossProfitMargin,operatingProfitMargin,pretaxProfitMargin,netProfitMargin,effectiveTaxRate,returnOnAssets,returnOnEquity,returnOnCapitalEmployed,netIncomePerEBT,ebtPerEbit,ebitPerRevenue,debtRatio,debtEquityRatio,longTermDebtToCapitalization,totalDebtToCapitalization,cashFlowToDebtRatio,companyEquityMultiplier,fixedAssetTurnover,assetTurnover,operatingCashFlowSalesRatio,freeCashFlowOperatingCashFlowRatio,cashFlowCoverageRatios,shortTermCoverageRatios,capitalExpenditureCoverageRatio,dividendPaidAndCapexCoverageRatio,dividendPayoutRatio,priceBookValueRatio,priceToBookRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,enterpriseValueMultiple,priceFairValue,revenueGrowth,grossProfitGrowth,ebitgrowth,operatingIncomeGrowth,netIncomeGrowth,epsgrowth,epsdilute